# Make a simple FeedForward network for the bus data

## Initialization

In [1]:
import sys
sys.path.append('../misc')

from MoviaBusDataset import MoviaBusDataset
from BaseNetwork import BaseNetwork
import torch
from torch.nn import Linear, Sequential, ReLU, L1Loss
from torch.optim import Adam, lr_scheduler
import numpy as np

## Import data using the MoviaBusDataset

In [2]:
previous_timesteps = 6
prediction_steps = 3
batch_size = 25
num_epochs = 100

train = MoviaBusDataset('../data/train', interpolation=True, prev_timesteps=previous_timesteps, max_future_time_steps=prediction_steps, timeofday = True)
test = MoviaBusDataset('../data/test', interpolation=True, prev_timesteps=previous_timesteps, max_future_time_steps=prediction_steps, timeofday = True)

train.normalize()
test.normalize(train.mean, train.std)

../misc\MoviaBusDataset.py:117: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat((dataframe.drop('TimeOfDay', axis=1) for dataframe in self.dataframes))


## The Model

In [3]:
class FNN(BaseNetwork):
    def __init__(self, num_hidden):
        super().__init__()
        

        self.FNN = Sequential(
            Linear(previous_timesteps+1, num_hidden),
            ReLU(),
            Linear(num_hidden, num_hidden),
            ReLU(),
            Linear(num_hidden, 1),
        )

    def forward(self,x):
        """
        x : [batch_size, prev_timesteps, num_roads]
        
        """
        #Transpose input, such that the previous time steps are the last dimension
        x = x.transpose(2,1)

        predictions = []
        for _ in range(self.max_timestep):
            #Run the input through the network
            prediction = self.FNN(x).squeeze()

            #Append the prediction to the list of predictions. 
            #If the data includes timeofday, this shouldn't be included
            predictions.append(prediction[:,:self.num_roads])

            #remove oldest timestep
            x = x[:,:,1:]
            #unsqueeze output so its size is [batch_size, num_roads, timesteps]
            prediction = prediction.unsqueeze(2)

            #append the new prediction to the input
            x = torch.cat((x,prediction),dim=2)


        return torch.stack(predictions,1)

net = FNN(num_hidden=100)

optimizer_fun = lambda param : Adam(param, lr=1e-2, weight_decay=1e-4)
scheduler_fun = lambda optim : lr_scheduler.StepLR(optim, step_size=10, gamma=0.1)
criterion = L1Loss()

In [4]:
train[2]

{'data': tensor([[ 0.6723, -0.1816,  0.3230,  ...,  0.7296, -0.5585,  0.2803],
         [-1.4166, -0.4299,  1.5753,  ...,  0.6366,  1.4346,  0.2841],
         [-0.2761,  0.8878, -0.7089,  ...,  0.3768,  0.5518,  0.2879],
         ...,
         [-0.9434,  0.8483,  0.4280,  ..., -0.7739,  1.5855,  0.2955],
         [ 0.5828,  1.1569, -0.3593,  ..., -0.4041, -0.0994,  0.2992],
         [ 0.6052,  0.9525, -0.2314,  ..., -0.0342, -1.7842,  0.3030]]),
 'target': tensor([[ 0.8600,  0.5668, -0.6363, -0.3945, -0.6439, -0.0745,  0.4580,  0.0083,
          -0.7075, -4.9371,  0.4238, -0.7538,  0.1196, -1.3977,  0.1408,  0.4247,
           0.0908,  0.3790, -0.2577, -0.1980,  0.5854, -1.4076,  0.1407, -0.9605,
          -0.0942, -0.4990,  0.0088,  0.7225,  0.4802,  0.4723,  0.2451,  0.8413,
           2.2939,  1.8249, -0.3864,  1.8959,  2.2265,  0.7014,  2.1136,  0.9435,
          -1.3905,  0.8391, -0.7296,  1.8560, -0.6583,  0.2330, -0.6163, -0.2604,
           0.6972, -0.7223, -0.6802, -0.6448, -0

## Training the network

In [5]:
net.train_network(train, test,batch_size=batch_size, 
                  num_epochs=30,
                  optimizer_fun=optimizer_fun,
                  scheduler_fun=scheduler_fun,
                  criterion=criterion)

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 3 and 2 in dimension 1 at c:\programdata\miniconda3\conda-bld\pytorch-cpu_1532498166916\work\aten\src\th\generic/THTensorMath.cpp:3616

## Evaluate the network on a test dataset 

In [ ]:
net.get_MAE_score(timestep=1)

### Example of modelled data vs real data for one road segment 

In [ ]:
%matplotlib notebook

net.visualize_road(timesteps=1, road=23)

In [ ]:
train.dataframes[0].index[0:10].values.TotalSeconds()

In [ ]:
import datetime
train.dataframes[0].index[0:10].values.astype(datetime.datetime)

In [ ]:
train.dataframes[0].index[0:10].values